In [1]:
import pandas as pd
import numpy as np
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM,Dense,Embedding,Input,Dropout
from  keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras import optimizers
import keras
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
import os
import time
from sklearn.utils import shuffle

Using TensorFlow backend.


In [2]:
dirs = os.listdir('./aclImdb/train')

In [3]:
text = []
labels = []
for i in dirs:
    if i == 'neg':
        for j in os.listdir('./aclImdb/train/neg'):
            with open('./aclImdb/train/neg/'+j, 'rb') as f:
                text.append(f.read())
                labels.append([1,0])
    if i == 'pos':
        for j in os.listdir('./aclImdb/train/pos'):
            with open('./aclImdb/train/pos/'+j, 'rb') as f:
                text.append(f.read())
                labels.append([0,1])

In [4]:
my_set = set()
for x in text:
    for i in str(x).split(' '):
        my_set.add(i)

In [5]:
len(my_set)

295309

In [6]:
word_dict = {}
for index, word in enumerate(my_set):
    word_dict[word] = index

In [7]:
def encode(text):
    vector = []
    for word in text.split(' '):
        vector.append(word_dict[word])
    return vector

In [9]:
#vector takes fixed lenght i/p so we pad remaining
reviews = []
for review in text[:1000]:
    reviews.append(encode(str(review)))

In [11]:
reviews = pad_sequences(reviews,maxlen=100,value=0.)

In [12]:
reviews[0]

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,  27789, 146859,  36222, 245350, 131211,
       117847, 124111,  64241, 162775, 122688, 251065, 184495, 204113,
        78894, 184495, 213035, 236372,  72582,  74826,  24162, 184495,
       266666, 200350,  66696, 281833, 262539, 121202, 131211, 122688,
       118827,  63056, 240682, 208631,  99749, 167929, 266666, 159925,
       146372, 246181,  92889,  41309], dtype=int32)

In [13]:
reviews, labels = shuffle(reviews, labels[:1000])

# LSTM

In [14]:
adam = optimizers.Adam(lr=0.01)
model = Sequential()
model.add(Embedding(len(my_set)+1,64,input_length=100))
model.add(LSTM(64))
model.add(Dense(2,activation='softmax'))

model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())
model.fit(reviews, np.array(labels[:1000]),batch_size=100,verbose=1,epochs=7)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 64)           18899840  
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 18,932,994
Trainable params: 18,932,994
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/7
1000/1000 [==============================] - 6s - loss: 0.1716 - acc: 0.9560     
Epoch 2/7
1000/1000 [==============================] - 6s - loss: 1.1921e-07 - acc: 1.0000     
Epoch 3/7
1000/1000 [==============================] - 

In [15]:
model.save('lstm_1.h5')